# This notebook incorporate the Killercopscanada data + a data from news artcles found from google

In [352]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateparser
import datetime
import os
import json
import re
import difflib
import string
from IPython.display import clear_output
from dateparser.search import search_dates
from dateparser_data.settings import default_parsers
parsers = [parser for parser in default_parsers if parser != 'relative-time']
# >>> parse('today', settings={'PARSERS': parsers})

can_province_names = {
      'Alberta':'AB',
      'British Columbia':'BC',
      'Manitoba':'MB',
      'New Brunswick':'NB',
      'Newfoundland and Labrador':'NL',
      'Newfoundland':'NL',
      'Nova Scotia':'NS',
      'Northwest Territories':'NT',
      'Nunavut':'NU',
      'Ontario':'ON',
      'Prince Edward Island':'PE',
      'Quebec':'QC',
      'Saskatchewan':'SK',
      'Yukon':'YT',
    }


can_province_keys = {
      'AB':'Alberta',
      'BC':'British Columbia',
      'MB':'Manitoba',
      'NB':'New Brunswick',
      'NL':'Newfoundland and Labrador',
      'NS':'Nova Scotia',
      'NT':'Northwest Territories',
      'NU':'Nunavut',
      'ON':'Ontario',
      'PE':'Prince Edward Island',
      'QC':'Quebec',
      'SK':'Saskatchewan',
      'YT':'Yukon',
    }


def Assign_Death_Code(df,Keys,Cols,Type,Recat=999,Validate = False,ValCols=None):
    for k in Keys:
        for c in Cols:
            if Validate == True:
                print(df.loc[((df['Death_Category']==Recat)&
                    (df[c].str.contains(k, flags=re.IGNORECASE, regex=True))),
                   ValCols].values)
            else:
                df.loc[((df['Death_Category']==Recat)&
                    (df[c].str.contains(k, flags=re.IGNORECASE, regex=True))),
                   'Death_Category']=Death_Catageory_Code[Type]
            
def FindDates(ss):
    flag = 'Parsed'
    
    DD = re.findall (r'\((.*?)\)',ss)
    for d in DD:
        if len(d)<=4:
            ss=ss.replace('('+d+')','')
    DD = re.findall(r'(?<=-)\w+',ss)
    for d in DD:
        ss = ss.replace('-'+d,'')
    DD = re.findall(r'(?<=/)\w+',ss)
    if len(DD)==1:
        ss = ss.replace('/'+DD[0],'')
    s = ss.replace('February 13, February 16,','February 13,').replace('(',' _ ').replace(')',' _ ').replace('.','').replace(',','').replace('$1500','')
    Result=search_dates(s,languages=['en'], settings={'PARSERS': parsers,
                                                   'PREFER_DAY_OF_MONTH': 'first',
                                                   'PREFER_DATES_FROM': 'past',
                                                   'REQUIRE_PARTS': ['month', 'year']
                                                      })
#     print(s,Result)
    if Result is not None:
        date = Result[-1][-1]
    else:
        date = None
        
    return(date,flag)
            
Death_Catageory_Code = {
    'Shooting':1,
#     'Rubber Bullet':2,
    'Taser':3,
    'Other Weapon':4,
    'Excited Delirium':9,
    'Use of Force':10,
    'Accidental':15,
    'Overdose/Intoxication':25,
    'Vehicle':50,
    'In Custody':75,
    'Suicide':100,
    'Other/Unknown':999
}
Death_Category_SearchKeys = {
    'Shooting':['shoot','shot ','gunshot','rubber bullet','fired'],
#     'Rubber Bullet':[],
    'Taser':['tase','taze','stun gun'],
    'Other Weapon':['baton','club','pepper spray','pepper-spray'],
    'Excited Delirium':['excited delirium'],    
    'Use of Force':['Physical force','restrain','subdue','struggling','struggled','resisted','altercation',
                    'broken rib','collapsed lung',"beat and kick",'dragged','held him down'
#                     'handcuff','apprehended',
#                     'arrested','Anoxic brain injury','interact','trauma','bleeding','blood','strangling',
#                     'head injury','collapsed lung','pushed','fight','resisting','kicked','altercation',
#                     'respiratory failure','injuries'
                   ],
    'Accidental':['Fall','Fell','jump','hypothermia','drowning','suffocat'],
    'Overdose/Intoxication':['Methamphetamine','Cocaine','methadone','alcohol','pills','drugs',
                 'intoxication','overdose','toxicity','blocked with vomit'],
    'Vehicle':['crash','vehicle',"hit by officer's car",'collision'],
    'In Custody':['cell','custody','hospital','Died of an allergy','Found unresponsive','Segregation'],
    'Suicide':['suicide','suicidal','hang','hung','self-inflicted','Incisions of neck'],
    'Other/Unknown':['Unknown']
}

# Month_Abb{}

# Killer Cops Canada Data
* I've scraped this wordpress blog and saved the data (see other notebook) and saved the data as .json files

In [353]:
Length = []
Date = []
TitleDate = []
TitleDate_Flag = []
Title = []
Summary = []
ID = []
# Keys = []
Prov = []
URL = []
Categories = []
for i in range (1,31):
    filename = "killercopscanada/page"+str(i)+".json"

    with open(filename) as json_file:
        json_data = json.load(json_file)
        for post in json_data['posts']: 
#             if post['ID']== 1535:#_KCC
            Date.append(post['date'])
            Length.append(len(post['content']))
            Title.append(post['title'])
            Summary.append(post['content'])
            ID.append(str(post['ID'])+'_KCC')
#             Keys.append(list(post['tags'].keys()))
            Tag_Cat = list(post['categories'].keys())+list(post['tags'].keys())
            Categories.append(list(dict.fromkeys(Tag_Cat))) ## Remove duplicates
#             matches = datefinder.find_dates(post['title'].replace('Sept.','Sep.'))
            URL.append(post['URL'])
#             print(post['title'])
            date,flag = FindDates(post['title'])
            TitleDate.append(date)
            TitleDate_Flag.append(flag)

KCC = pd.DataFrame(data = {'ID':ID,
                           'URL':URL,
                           'PostDate':Date,
                           'Title':Title,
                           'Categories':Categories,
#                            'Tags':Keys,
                           'PostLength':Length,
                           'Text':Summary,
                           'TitleDate':TitleDate,
                           'TitleDate_Flag':TitleDate_Flag})

KCC = KCC.set_index(pd.DatetimeIndex(KCC['PostDate']))
KCC['Match']=0
KCC.index=KCC.index.tz_localize(None)
# [1,1,1,1]
# KCC.loc[((KCC.index-KCC['TitleDate']<datetime.timedelta(-1))&(KCC['TitleDate_Flag']=='Found')),'TitleDate']=np.nan

In [354]:
# ss = 'July 7-8, 2020'
#     print(d)
#     if len(d)<=4:
#         ss=ss.replace('('+d+')',' ')

In [355]:
# print(KCC.loc[KCC['TitleDate_Flag']=='Missing'][-1:].values)
# print(KCC['Tags'])
KCC['prov']='Unspecified'

for i,row in KCC.iterrows():
    pc = 0
    for cat in row['Categories']:
        try:
            KCC.loc[KCC.index==i,'prov']=can_province_names[cat]
            pc += 1
        except:
            pass
    if pc > 1:
        KCC.loc[KCC.index==i,'prov']='Multiple'
# print(KCC['Categories'])
print(KCC.groupby('prov').count()['ID'])
# T = KCC.loc[KCC['ID']=='1039_KCC']
# print(T.index-T.TitleDate,T.TitleDate)
# print(KCC.loc[((KCC.index-KCC['TitleDate']<datetime.timedelta(-1))&(KCC['TitleDate_Flag']=='Found')),'TitleDate'])

prov
AB              89
BC              99
MB              42
Multiple         7
NB              13
NL               5
NS               8
NU               9
ON             193
PE               1
QC              96
SK              21
Unspecified      4
YT               2
Name: ID, dtype: int64


In [356]:
#Filter out lists in KCC to get just posts about incidents
print(KCC.shape)
Excluders=['C.R.A.P', 
 'Police-Involved Deaths in Canada in 2017:',
 'People who Died Following a Police Intervention Since 1987',
 'Activists Killed by Cops',
 'A Victim of Power',
 'Third Consecutive Yearly Increase in Police-Involved Deaths in BC',
 'Deadly Force',
 'Delayed Information, Counterinformation',
 'IIO Report Documents Police Contempt for Oversight Agencies',
 'Still No Oversight, Still No Accountability', 
 'A Troubled Explanation']
for ex in Excluders:
    print(ex)
    if ex == 'C.R.A.P':
        KCC = KCC.loc[KCC['Text'].str.contains(ex)==False]
    else:
        KCC = KCC.loc[KCC['Title'].str.contains(ex)==False]
    print(KCC.shape)

(589, 11)
C.R.A.P
(562, 11)
Police-Involved Deaths in Canada in 2017:
(561, 11)
People who Died Following a Police Intervention Since 1987
(560, 11)
Activists Killed by Cops
(552, 11)
A Victim of Power
(552, 11)
Third Consecutive Yearly Increase in Police-Involved Deaths in BC
(551, 11)
Deadly Force
(550, 11)
Delayed Information, Counterinformation
(549, 11)
IIO Report Documents Police Contempt for Oversight Agencies
(548, 11)
Still No Oversight, Still No Accountability
(547, 11)
A Troubled Explanation
(546, 11)


# Missing Values
* This was my first attempt at filling the gaps, maunually creating a new spreadhseet & adding records
* Many of the records were pulled from killercopscanada blog.
* Some others were added from other sources
    * Where KCC is he linked source, I'll add the text post as the text summary and add it to PID


In [357]:
MD = pd.read_csv('MissingRecords_Skeeter_2021.csv',parse_dates=['date'],index_col=['id_victim'])
MD = MD.fillna('')
# MD = MD.rename(columns={'Date':'date'})
# Find any records missing a summary and add it from the source
# I've already done for most records 
# print(MD[['summary','KCC_ID']])
for i, row in MD.loc[MD['data_source']!='killercopscanada'].iterrows():
    if row['summary'] == '':
        print(row['Link'])
        text = input('Enter Summary')
        MD.loc[MD.index==i,'summary']=text

# Any KCC posts, add the post text and URL
for i,row in MD.loc[MD['data_source']=='killercopscanada'].iterrows():
    if row ['summary'] == '':
        for l in row['Link'].split(' : '):
            if KCC.loc[KCC['URL']==l,'ID'].shape[0]>0:
    #             print()
                MD.loc[MD.index==i,'KCC_ID']=KCC.loc[KCC['URL']==l,'ID'].values[0]
                MD.loc[MD.index==i,'summary']=KCC.loc[KCC['URL']==l,'Text'].values[0]

# print(MD['summary'])
MD.to_csv('MissingRecords_Skeeter_2021.csv')
MD = MD.reset_index().set_index(pd.DatetimeIndex(MD.date))
MD['last_name']=MD['last_name'].replace({'Unknown':'Unspecified','':'Unspecified'})
MD['last_name']=MD['last_name'].fillna('Unspecified')
# MD['city_town']=MD['city_town'].fillna('')
# MD['city_town']
MD['data_source']='June'
MD.head()

,id_victim,date,first_name,last_name,middle_name,alias_nickname,age,gender,race,prov,department,cause_death,DEATH CATEGORY,data_source,Link,summary,name,KCC_ID
date,,,,,,,,,,,,,,,,,,
1981-03-29,MR_0,1981-03-29,Jeannette,Kelly,,,,Female,White,ON,RCMP,thrown off balcony,Off-Duty/Retired,June,https://en.wikipedia.org/wiki/Patrick_Kelly_(f...,"In 1984, Kelly was convicted of the first-degr...",Jeannette Kelly,
1990-11-25,MR_1,1990-11-25,Neil,Stonechild,,,17.0,Male,Indigenous,SK,Saskatoon Police Service,Starlight Tour,Other,June,https://en.wikipedia.org/wiki/Neil_Stonechild,"Neil Stonechild (August 24, 1973 – November 25...",Neil Stonechild,None
2000-01-29,MR_2,2000-01-29,Rodney,Naistus,,,25.0,Male,Indigenous,SK,Saskatoon Police Service,Starlight Tour,Other,June,https://en.wikipedia.org/wiki/Saskatoon_freezi...,Victims who died from hypothermia include Rodn...,Rodney Naistus,None
2000-02-03,MR_3,2000-02-03,Lawrence,Wegner,,,30.0,Male,Indigenous,SK,Saskatoon Police Service,Starlight Tour,Other,June,https://en.wikipedia.org/wiki/Saskatoon_freezi...,Victims who died from hypothermia include Rodn...,Lawrence Wegner,None
2001-06-26,MR_4,2001-06-26,Lucie,Gelinas,,,,Female,Unspecified,QC,RCMP,Gunshot,Off-Duty/Retired,June,https://www.cbc.ca/news/canada/ex-rcmp-officer...,A former RCMP marksman who shot and killed an ...,Lucie Gelinas,None


In [358]:
PID = pd.read_csv('CombinedSourceData.csv',parse_dates=['date'])

# print(PID.groupby('last_name').count()['id_victim'].sort_values())
# PID.loc[PID['last_name']=='Gagnon']
# PID.head()
print(PID.loc[((PID['last_name']=='Williams')&(PID['prov']=='BC')),['id_victim','name','race','city_town','data_source','postal_code','address_intersection']])


    id_victim                       name        race  \
246   0112-V2             Jovan Williams  Indgienous   
247   0112-V1  Shirley Beatrice Williams  Indgienous   

                    city_town          data_source postal_code  \
246  Granisle;    Unspecified  Deadly Force (2018)     V0J 1W0   
247  Granisle;    Unspecified  Deadly Force (2018)     V0J 1W0   

    address_intersection  
246         Morrison St.  
247         Morrison St.  


In [359]:
PID = pd.read_csv('CombinedSourceData.csv',parse_dates=['date'])
# print(PID.head())
PID = PID.reset_index(drop=True).set_index(pd.DatetimeIndex(PID.date),drop=True)
PID['last_name']=PID['last_name'].fillna('Unspecified')
# print(PID.head())

# Check if there are any "same" names within a 60 day window, allowing for typos.
# Check for missing names within a shorter widow, didnt find any tho
# Add any other records
# PID = PID.copy()
# Window_named = 60
# Window_Unnamed = 5
Window_d = 4
Thresh = .85
# c = 0
# MD['Append'] = 1
MD['MD_match'] = 'None'
PID['MD_match'] = 'None'
# PID['CBC_id'] 
print(PID.count()['name'])
for i,row in MD.iterrows():
    Subset = PID.loc[((np.abs(PID.index-i)<=datetime.timedelta(Window_d))&
                           (PID['prov']==row['prov']))]
    if Subset.shape[0]>0:
        if row['last_name']!='Unspecified':
            Match1=difflib.get_close_matches(row['last_name'],Subset['last_name'], cutoff=Thresh)
#             print(row['last_name'],Subset['last_name'])
            if len(Match1)>1:
                Match2 = difflib.get_close_matches(row['first_name'],Subset.loc[Subset['last_name']==Match1[0],
                                                                                'first_name'], cutoff=Thresh-.2)
                S = Subset.loc[((Subset['last_name']==Match1[0])&(Subset['first_name']==Match2[0])),'id_victim'].values[0]
                PID.loc[PID['id_victim']==S,'MD_match'] = row['id_victim']
                MD.loc[MD['id_victim']==row['id_victim'],'MD_match']=Subset.loc[((Subset['last_name']==Match1[0])&(Subset['first_name']==Match2[0])),
                                                                                'id_victim'].values[0]
            if len(Match1)==1:
                
                PID.loc[PID['id_victim']==Subset.loc[Subset['last_name']==Match1[0],'id_victim'].values[0],
                        'MD_match'] = row['id_victim']
                MD.loc[MD['id_victim']==row['id_victim'],'MD_match']=Subset.loc[Subset['last_name']==Match1[0],
                                                                                'id_victim'].values[0]
        else:
            Subset = Subset.loc[Subset['age']==row['age']]
            if Subset.loc[Subset['prov']==row['prov']].shape[0]==1:
                
#                 print(Subset.age,row['age'])
                PID.loc[PID['id_victim']==Subset.loc[Subset['prov']==row['prov'],'id_victim'].values[0],'MD_match'] = row['id_victim']
                MD.loc[MD['id_victim']==row['id_victim'],'MD_match']=Subset.loc[Subset['prov']==row['prov'],'id_victim'].values[0]
#             else:   
#                 print()#'XXXXX')
# #                 print(row)
#                 print(Subset.index-i,Subset[['name','city_town','age']],row['summary'])
#                 print(Subset['department'],row['department'])
                
#                 print(row[['name','department','prov']])
print(MD.shape[0])
print(PID.shape[0])
PID = PID.append(MD)
print(PID['data_source'])
print(PID.shape[0])
# PID2 = PID.copy()
MD_keep=['age','gender','race','cause_death','DEATH CATEGORY','Link','KCC_ID','department']
Drop = []
for i,row in PID.loc[((PID['MD_match']!='None'))].iterrows():
    Match = PID.loc[PID['id_victim']==row['MD_match']]
    if row['id_victim'].split('_')[0]!='MR':
        Match = PID.loc[PID['id_victim']==row['MD_match']]
        Match1=difflib.get_close_matches(row['last_name'],Match['last_name'], cutoff=Thresh)
        if len(Match1)>0:
            PID.loc[PID['id_victim']==row['id_victim'],'summary'] += ';    '+Match.loc[Match['id_victim']!=row['id_victim'],'summary'].iloc[0]
            if row['data_source']=='la CRAP (2021)':
#                 print('poop')
                PID.loc[PID['id_victim']==row['id_victim'],MD_keep] = Match.loc[Match['id_victim']!=row['id_victim'],
                                                          MD_keep].iloc[0].values
#                 print(PID.loc[PID['id_victim']==row['id_victim'],MD_keep])
            if Match['prov'][0]!=row['prov']:
                PID.loc[PID['id_victim']==row['id_victim'],
                        'prov'] += ';    '+Match.loc[Match['id_victim']!=row['id_victim'],
                                                          'prov'].iloc[0]
#             PID2 = PID2.loc[PID2['id_victim']!=Match['id_victim'].values[0]]
            Drop.append(Match['id_victim'].values[0])
        elif i - Match.index<=datetime.timedelta(Window_d):
#             PID.loc[PID['id_victim']==row['id_victim'],'summary'] += ';    '+Match.loc[Match['id_victim']!=row['id_victim'],'summary'].iloc[0]
#             if row['data_source']=='la CRAP (2021)':
#                 PID.loc[PID['id_victim']==row['id_victim'],MD_keep] = Match.loc[Match['id_victim']!=row['id_victim'],
#                                                           MD_keep].iloc[0].values
# #                 print(PID.loc[PID['id_victim']==row['id_victim'],MD_keep])
#             if Match['prov'][0]!=row['prov']:
#                 PID.loc[PID['id_victim']==row['id_victim'],'Check'] = 'Yes'
# #                 print(Match.loc[Match['id_victim']!=row['id_victim']].iloc[0])
#                 PID.loc[PID['id_victim']==row['id_victim'],
#                         'prov'] += ';    '+Match.loc[Match['id_victim']!=row['id_victim'],
#                                                           'prov'].iloc[0]
            
    
    
            Drop.append(Match['id_victim'].values[0])
#             PID2 = PID2.loc[PID2['id_victim']!=Match['id_victim'].values[0]]
        else:
            print(i - Match.index)
# print(PID2.shape[0])
# PID = PID2.copy()
# PID = PID.drop(Drop)
PID = PID.loc[PID['id_victim'].isin(Drop)==False]


print(PID.groupby('data_source').count()['name'])
# print(PID2.groupby('data_source').count()['name'])
PID.head()

1501
229
1501
date
2012-01-06    Deadly Force (2018)
2010-08-25    Deadly Force (2018)
2007-11-20    Deadly Force (2018)
2008-07-31    Deadly Force (2018)
2008-06-27    Deadly Force (2018)
                     ...         
2021-05-19                   June
2021-05-20                   June
2021-05-22                   June
2021-05-25                   June
2020-10-08                   June
Name: data_source, Length: 1730, dtype: object
1730
data_source
Deadly Force (2018)    462
Deadly Force (2020)     97
Georgia St (2017)      107
June                   161
la CRAP (2021)         835
Name: name, dtype: int64


,id_victim,date,id_incident,day_week,prov,city_town,address_intersection,postal_code,location_type,first_name,...,data_source,Check,ds_rank,GS_match,Inquest date,laCRAP_match,MD_match,DEATH CATEGORY,Link,KCC_ID
date,,,,,,,,,,,,,,,,,,,,,
2012-01-06,0001-V1,2012-01-06,1.0,Fri,QC,Montréal; Montreal,1000 rue De La Gaucheti√®re,H3B 4W5,Urban,Farshad,...,Deadly Force (2018),Yes,0.0,None,NaN,49_CRAP,None,NaN,NaN,NaN
2010-08-25,0338-V1,2010-08-25,338.0,Wed,ON,Brantford; Unspecified,91A Sympatica,N3P 1X6,Urban,Evan,...,Deadly Force (2018),Yes,0.0,None,NaN,694_CRAP,None,NaN,NaN,NaN
2007-11-20,0337-V1,2007-11-20,337.0,Tue,ON,Kitchener; Unspecified,1400 Ottawa St. S,N2E 4E2,Urban,Trevor,...,Deadly Force (2018),Yes,0.0,None,NaN,678_CRAP,None,NaN,NaN,NaN
2008-07-31,0336-V1,2008-07-31,336.0,Thu,BC,Surrey; Unspecified,106th Ave. & 132 St.,V3T 3V9,Urban,Brandon,...,Deadly Force (2018),Yes,0.0,47_GS,NaN,1031_CRAP,None,NaN,NaN,NaN
2008-06-27,0335-V1,2008-06-27,335.0,Fri,BC,Surrey,72nd Ave. & 134th St.,V3W 2N5,Urban,Ryan,...,Deadly Force (2018),No,0.0,46_GS,NaN,None,None,NaN,NaN,NaN


In [360]:
# print(PID.loc[PID.last_name=='Diggle',MD_keep].values)
# # print(PID2.loc[PID2.last_name=='Diggle',['MD_match','department']])
# print(PID.columns)
# print(PID.loc[PID.id_victim=='MR_62'])

In [361]:
# print(PID.loc[PID.index])
PID.loc[PID['KCC_ID']=='None','KCC_ID']=np.nan
PID.loc[PID['KCC_ID']=='','KCC_ID']=np.nan
print(PID.groupby('KCC_ID').count()['prov'].sort_values())
# print(KCC.loc[KCC['ID']=='993_KCC'])

# PID = PID.reset_index(drop=True).set_index(pd.DatetimeIndex(PID.Date))
PID['KCC_ID']=PID['KCC_ID'].fillna('')
Exclude = PID.loc[((PID['KCC_ID']!='')),
              'KCC_ID'].values
# print(Exclude)
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
STOPWORDS.add('man')

# table = str.maketrans("","")
for i,row in KCC.iterrows():
    T=row['Title'].lower().translate(str.maketrans('', '', string.punctuation)).split()
#     T = T.
#     print(T)
    List =[]
    for word in T:
        if word not in STOPWORDS:
            List.append(word)
#         List.append
    S = set(List)
#     print(S)
    KCC.loc[KCC.index==i,'JustWords_Title']=[S]

KCC_ID
1009_KCC     1
1729_KCC     1
1721_KCC     1
1706_KCC     1
1703_KCC     1
            ..
1155_KCC     2
1151_KCC     2
1279_KCC     2
1217_KCC     2
993_KCC     27
Name: prov, Length: 130, dtype: int64


In [362]:
# string.punctuation

In [363]:

PID['middle_name']=PID['middle_name'].fillna('')
KCC['id_victim']=''
KCC['Match']=0.0
KCC['name']=''
Dated = KCC.loc[((KCC['TitleDate'].isnull()==False)&(KCC['ID'].isin(Exclude)==False))].copy()
# print(KCC.shape)
print(Dated.shape)
# print(NotDated.shape)
# PID = PID.copy()#loc[PID.name!='Unspecified']

# for 
Window_d = 4
Thresh = .85
Dated = Dated.reset_index(drop=True).set_index(pd.DatetimeIndex(Dated.TitleDate))
# print(Dated.columns)
for i,row in PID.iterrows():#.loc[PID['id_victim']=='MR_205']
    Match = Dated.loc[np.abs(Dated.index-i)<=datetime.timedelta(Window_d)].copy()
    if Match.shape[0]>0 and row['name']!='Unspecified':
        
        LN=row['last_name'].lower()
        Match['match'] = Match.JustWords_Title.apply(lambda words: difflib.get_close_matches(LN,words, cutoff=Thresh))
        
        if Match.shape[0]>0:#.loc[Match['match'].str.len()!=0]
            for j,row2 in Match.loc[Match['match'].str.len()!=0].iterrows():
                PID.loc[PID.id_victim==row['id_victim'],'KCC_ID'] += ' ' + row2['ID']
                PID.loc[PID.id_victim==row['id_victim'],'summary'] += '       ' + row2['Text']
                Dated.loc[Dated['ID']==row2['ID'],'Match']+=1
                Dated.loc[Dated['ID']==row2['ID'],'id_victim']+=';'+row['id_victim']
                Dated.loc[Dated['ID']==row2['ID'],'name']+=';'+row['name']
#         else:
            for j,row2 in Match.loc[Match['match'].str.len()==0].iterrows():
                if can_province_keys[row['prov']] in row2['Categories']:
                    PID.loc[PID.id_victim==row['id_victim'],'KCC_ID'] += ' ***' + row2['ID']
                    PID.loc[PID.id_victim==row['id_victim'],'summary'] += '       ***' + row2['Text']
                    Dated.loc[Dated['ID']==row2['ID'],'Match']+=.0001
                    Dated.loc[Dated['ID']==row2['ID'],'id_victim']+=';**'+row['id_victim']
                    Dated.loc[Dated['ID']==row2['ID'],'name']+=';**'+row['name']

    elif Match.shape[0]>0:
        for j,row2 in Match.iterrows():
            if can_province_keys[row['prov']] in row2['Categories']:
                PID.loc[PID.id_victim==row['id_victim'],'KCC_ID'] += ' **' + row2['ID']
                PID.loc[PID.id_victim==row['id_victim'],'summary'] += '       **' + row2['Text']
                Dated.loc[Dated['ID']==row2['ID'],'Match']+=.01
                Dated.loc[Dated['ID']==row2['ID'],'id_victim']+=';**'+row['id_victim']
                Dated.loc[Dated['ID']==row2['ID'],'name']+=';**'+row['name']
                
NotDated = KCC.loc[((KCC['TitleDate'].isnull()==True)&(KCC['ID'].isin(Exclude)==False))].copy()
c = 0
NotDated = NotDated.append(Dated.loc[Dated['Match']==0])
NotDated = NotDated.set_index(pd.DatetimeIndex(NotDated['PostDate']).tz_localize(None))

Window_d = 10
for i,row in PID.iterrows():#.loc[PID['id_victim']=='0144-V1']
    Match = NotDated.loc[((NotDated.index-i<=datetime.timedelta(Window_d))&(NotDated.index-i>=datetime.timedelta(0)))].copy()
    if Match.shape[0]>0 and row['name']!='Unspecified':
        LN=row['last_name'].lower()
        Match['match'] = Match.JustWords_Title.apply(lambda words: difflib.get_close_matches(LN,words, cutoff=.7))
        
        if Match.shape[0]>0:
            for j,row2 in Match.loc[Match['match'].str.len()!=0].iterrows():
                PID.loc[PID.id_victim==row['id_victim'],'KCC_ID'] += ' ' + row2['ID']
                PID.loc[PID.id_victim==row['id_victim'],'summary'] += '       ' + row2['Text']
                NotDated.loc[NotDated['ID']==row2['ID'],'Match']+=1
                NotDated.loc[NotDated['ID']==row2['ID'],'id_victim']+=';'+row['id_victim']
                NotDated.loc[NotDated['ID']==row2['ID'],'name']+=';'+row['name']
            for j,row2 in Match.loc[Match['match'].str.len()==0].iterrows():
                if can_province_keys[row['prov']] in row2['Categories']:
                    PID.loc[PID.id_victim==row['id_victim'],'KCC_ID'] += ' ***' + row2['ID']
                    PID.loc[PID.id_victim==row['id_victim'],'summary'] += '       ***' + row2['Text']
                    NotDated.loc[NotDated['ID']==row2['ID'],'Match']+=.0001
                    NotDated.loc[NotDated['ID']==row2['ID'],'id_victim']+=';**'+row['id_victim']
                    NotDated.loc[NotDated['ID']==row2['ID'],'name']+=';**'+row['name']

    elif Match.shape[0]>0:
        for j,row2 in Match.iterrows():
            if can_province_keys[row['prov']] in row2['Categories']:
                PID.loc[PID.id_victim==row['id_victim'],'KCC_ID'] += ' **' + row2['ID']
                PID.loc[PID.id_victim==row['id_victim'],'summary'] += '       **' + row2['Text']
                NotDated.loc[NotDated['ID']==row2['ID'],'Match']+=.01
                NotDated.loc[NotDated['ID']==row2['ID'],'id_victim']+=';**'+row['id_victim']
                NotDated.loc[NotDated['ID']==row2['ID'],'name']+=';**'+row['name']

    if row['name']!='Unspecified':
        LN=row['last_name'].lower()
        NotDated['match'] = NotDated.JustWords_Title.apply(lambda words: difflib.get_close_matches(LN,words, cutoff=.9))
        
        for j,row2 in NotDated.loc[NotDated['match'].str.len()!=0].iterrows():
            if can_province_keys[row['prov']] in row2['Categories'] or row['prov'] == row2['prov']:
                PID.loc[PID.id_victim==row['id_victim'],'KCC_ID'] += ' *' + row2['ID']
                PID.loc[PID.id_victim==row['id_victim'],'summary'] += '       *' + row2['Text']
                NotDated.loc[NotDated['ID']==row2['ID'],'Match']+=.1
                NotDated.loc[NotDated['ID']==row2['ID'],'id_victim']+=';**'+row['id_victim']
                NotDated.loc[NotDated['ID']==row2['ID'],'name']+=';**'+row['name']
#         else:

print()
print('Dated -')
print('Matched: ',Dated.loc[Dated['Match']>0].shape[0])
print('NotDated -')
print('Matched: ',NotDated.loc[NotDated['Match']>0].shape[0])
print('Not Matched: ',NotDated.loc[NotDated['Match']==0].shape[0])  
PID.head()

(217, 14)

Dated -
Matched:  214
NotDated -
Matched:  176
Not Matched:  28


,id_victim,date,id_incident,day_week,prov,city_town,address_intersection,postal_code,location_type,first_name,...,data_source,Check,ds_rank,GS_match,Inquest date,laCRAP_match,MD_match,DEATH CATEGORY,Link,KCC_ID
date,,,,,,,,,,,,,,,,,,,,,
2012-01-06,0001-V1,2012-01-06,1.0,Fri,QC,Montréal; Montreal,1000 rue De La Gaucheti√®re,H3B 4W5,Urban,Farshad,...,Deadly Force (2018),Yes,0.0,None,NaN,49_CRAP,None,NaN,NaN,
2010-08-25,0338-V1,2010-08-25,338.0,Wed,ON,Brantford; Unspecified,91A Sympatica,N3P 1X6,Urban,Evan,...,Deadly Force (2018),Yes,0.0,None,NaN,694_CRAP,None,NaN,NaN,
2007-11-20,0337-V1,2007-11-20,337.0,Tue,ON,Kitchener; Unspecified,1400 Ottawa St. S,N2E 4E2,Urban,Trevor,...,Deadly Force (2018),Yes,0.0,None,NaN,678_CRAP,None,NaN,NaN,
2008-07-31,0336-V1,2008-07-31,336.0,Thu,BC,Surrey; Unspecified,106th Ave. & 132 St.,V3T 3V9,Urban,Brandon,...,Deadly Force (2018),Yes,0.0,47_GS,NaN,1031_CRAP,None,NaN,NaN,
2008-06-27,0335-V1,2008-06-27,335.0,Fri,BC,Surrey,72nd Ave. & 134th St.,V3W 2N5,Urban,Ryan,...,Deadly Force (2018),No,0.0,46_GS,NaN,None,None,NaN,NaN,


In [364]:
print(NotDated.loc[NotDated['Match']==0,['ID','Title','TitleDate']].values)
NotDated['match'] = NotDated.JustWords_Title.apply(lambda words: difflib.get_close_matches('vienneau',words, cutoff=.85))
print(NotDated.loc[NotDated['match'].str.len()>=1,['Title','ID']].values)
PID = PID.reset_index(drop=True).set_index('id_victim')

[['937_KCC'
  'Killer Toronto Cop James Forcillo Charged With Breaching Bail Conditions'
  NaT]
 ['906_KCC'
  'Trial for Killer Cop Simon Beaulieu Hears Cops Should Be Given More Consideration Than “Regular” Citizens'
  NaT]
 ['899_KCC'
  'Letter From SIU Head to Ottawa Police Chief Raises Questions About Transparency and “Old Boy” Favors in Investigations'
  NaT]
 ['888_KCC'
  'Killer Cop Simon Beaulieu and the Policing Assumptions that Take Lives'
  NaT]
 ['863_KCC'
  'Killer Toronto Cop James Forcillo’s Appeal Begins: Seeks Bogus Suicide-By-Cop Excuse'
  NaT]
 ['817_KCC'
  'ASIRT Uses Bogus “Excited Delirium” Again To Excuse Edmonton Police Taser Killing'
  NaT]
 ['811_KCC' 'Fifteen-Year-Old Shot by Peel Regional Police Dies' NaT]
 ['806_KCC'
  'IIU Finds Man Died in Custody After Winnipeg Police Use of Force' NaT]
 ['774_KCC'
  'Alberta Killer Cop Michelle Phillips Has First Court Appearance, Victim’s Family Not Notified'
  NaT]
 ['721_KCC'
  'Absurd Identity Confusion as SIU Lets 

# Find Matching records in KCC
* This is the next step, match up existing recrods.  Append missing.

In [365]:

print(PID.columns)

PID['Check']='Yes'
PID['postal_code']=PID['postal_code'].fillna('Unspecified')
PID.loc[(PID['postal_code']!='Unspecified'),'Check']='No'
Summary = PID[['summary','data_source']]
Summary['extra_source'] = ''
Summary.to_csv('Compiled/Summary.csv')
print(Summary.index)
Location = PID[['prov', 'city_town','address_intersection', 'postal_code','Check']]
Location.to_csv('Compiled/Location.csv')
Victim = PID[['name','first_name', 'middle_name', 'last_name', 'alias_nickname', 'age',
       'gender', 'race', 'ethnic_ancestry']]
Victim.to_csv('Compiled/Victim.csv')
Incident = PID[['date','armed_type', 'cause_death','DEATH CATEGORY', 'taser_deployed', 'injured_officer',
       'excited_delirium', 'mentral_distress_disorder', 'substance_abuse','department',
        'charge_type', 'charges', 'officers involved','data_source', 'Inquest date']]

Incident.to_csv('Compiled/Incident.csv')
PID.to_csv('Compiled/PID.csv')
# print(PID
PID.head()

Index(['date', 'id_incident', 'day_week', 'prov', 'city_town',
       'address_intersection', 'postal_code', 'location_type', 'first_name',
       'middle_name', 'last_name', 'alias_nickname', 'age', 'gender', 'race',
       'ethnic_ancestry', 'immigrant_refugee_naturalized', 'armed_type',
       'cause_death', 'taser_deployed', 'injured_officer', 'excited_delirium',
       'mentral_distress_disorder', 'substance_abuse', 'summary', 'department',
       'charge_type', 'charges', 'officers involved', 'name', 'data_source',
       'Check', 'ds_rank', 'GS_match', 'Inquest date', 'laCRAP_match',
       'MD_match', 'DEATH CATEGORY', 'Link', 'KCC_ID'],
      dtype='object')


<ipython-input-365-745e38072021>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Summary['extra_source'] = ''


Index(['0001-V1', '0338-V1', '0337-V1', '0336-V1', '0335-V1', '0334-V1',
       '0333-V1', '0332-V1', '0331-V1', '0330-V1',
       ...
       'MR_218', 'MR_219', 'MR_220', 'MR_221', 'MR_222', 'MR_223', 'MR_224',
       'MR_225', 'MR_226', 'MR_227'],
      dtype='object', name='id_victim', length=1662)


,date,id_incident,day_week,prov,city_town,address_intersection,postal_code,location_type,first_name,middle_name,...,data_source,Check,ds_rank,GS_match,Inquest date,laCRAP_match,MD_match,DEATH CATEGORY,Link,KCC_ID
id_victim,,,,,,,,,,,,,,,,,,,,,
0001-V1,2012-01-06,1.0,Fri,QC,Montréal; Montreal,1000 rue De La Gaucheti√®re,H3B 4W5,Urban,Farshad,,...,Deadly Force (2018),No,0.0,None,NaN,49_CRAP,None,NaN,NaN,
0338-V1,2010-08-25,338.0,Wed,ON,Brantford; Unspecified,91A Sympatica,N3P 1X6,Urban,Evan,,...,Deadly Force (2018),No,0.0,None,NaN,694_CRAP,None,NaN,NaN,
0337-V1,2007-11-20,337.0,Tue,ON,Kitchener; Unspecified,1400 Ottawa St. S,N2E 4E2,Urban,Trevor,,...,Deadly Force (2018),No,0.0,None,NaN,678_CRAP,None,NaN,NaN,
0336-V1,2008-07-31,336.0,Thu,BC,Surrey; Unspecified,106th Ave. & 132 St.,V3T 3V9,Urban,Brandon,Norman,...,Deadly Force (2018),No,0.0,47_GS,NaN,1031_CRAP,None,NaN,NaN,
0335-V1,2008-06-27,335.0,Fri,BC,Surrey,72nd Ave. & 134th St.,V3W 2N5,Urban,Ryan,Alexander,...,Deadly Force (2018),No,0.0,46_GS,NaN,None,None,NaN,NaN,


In [366]:
PID[PID.index.duplicated()]
# PID = PID.reset_index(drop=True).set_index(pd.DatetimeIndex(PID.Date))
# print(PID.loc[((PID['prov']=='BC')&(PID.index.month==5)&(PID.index.year==2021))])
# # print(PID.loc[PID['KCC_ID']!='Unspecified','prov'].count())


,date,id_incident,day_week,prov,city_town,address_intersection,postal_code,location_type,first_name,middle_name,...,data_source,Check,ds_rank,GS_match,Inquest date,laCRAP_match,MD_match,DEATH CATEGORY,Link,KCC_ID
id_victim,,,,,,,,,,,,,,,,,,,,,


In [367]:
print(KCC.loc[KCC['ID']=='618_KCC',['Title','TitleDate']].values)
s = 'IIO Investigates as Victoria Woman Dies Only Hours After Interaction with Police'
Result=search_dates(s,languages=['en'], settings={'PARSERS': parsers,
                                                   'PREFER_DAY_OF_MONTH': 'first',
                                                   'PREFER_DATES_FROM': 'past',
                                                   'REQUIRE_PARTS': ['month', 'year']
                                                      })
print(Result,parsers)

[['IIO Investigates as Victoria Woman Dies Only Hours After Interaction with Police'
  NaT]]
None ['timestamp', 'custom-formats', 'absolute-time']


In [368]:
PID['Check']=True
PID.loc[(PID['postal_code']!='Unspecified'),'Check']=False

PID.index=PID.index.rename('Date')
PID.to_csv('Compiled/Combined_Data.csv')
print(PID.columns)

Index(['date', 'id_incident', 'day_week', 'prov', 'city_town',
       'address_intersection', 'postal_code', 'location_type', 'first_name',
       'middle_name', 'last_name', 'alias_nickname', 'age', 'gender', 'race',
       'ethnic_ancestry', 'immigrant_refugee_naturalized', 'armed_type',
       'cause_death', 'taser_deployed', 'injured_officer', 'excited_delirium',
       'mentral_distress_disorder', 'substance_abuse', 'summary', 'department',
       'charge_type', 'charges', 'officers involved', 'name', 'data_source',
       'Check', 'ds_rank', 'GS_match', 'Inquest date', 'laCRAP_match',
       'MD_match', 'DEATH CATEGORY', 'Link', 'KCC_ID'],
      dtype='object')


In [369]:
print(PID.loc[PID['name']=='Juan Carlos Salvany Sailes','cause_death'].values)
# print(PID)

['Cardiac arrest / self-inflicted stab incise woundsof neck and chest']


In [370]:
print(PID.race.unique())

['Other' 'White' 'Unspecified' 'Indigenous' 'Latin American' 'Black'
 'Asian' 'South Asian' 'Middle Eastern' 'Indgienous' nan]
